In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

def get_mnist_loaders(train_batch_size, test_batch_size):
    """Get MNIST data loaders"""
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=train_batch_size, shuffle=True)

    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=test_batch_size, shuffle=True)

    return train_loader, test_loader

In [ ]:
class Net(nn.Module):
    def __init__(self, activation):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        self.activation = activation

    def forward(self, x):
        x = self.activation(self.conv1(x))
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.activation(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
def train(log_interval, model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        loss = F.nll_loss(output, target.to(device))
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data.to(device))
            test_loss += F.nll_loss(output, target.to(device), reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.to(device).view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * (correct / len(test_loader.dataset))

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    return test_accuracy


In [ ]:
use_cuda = torch.cuda.is_available()
print('Using PyTorch version:', torch.__version__, 'CUDA:', use_cuda)
device = torch.device("cuda" if use_cuda else "cpu")
device

Using PyTorch version: 1.12.1+cu113 CUDA: False


device(type='cpu')

In [ ]:
def train_mnist():

  cfg = { 'device' : "cuda" if torch.cuda.is_available() else "cpu",
          'train_batch_size' : 64,
          'test_batch_size' : 1000,
          'n_epochs' : 1,
          'seed' : 0,
          'log_interval' : 100,
          'save_model' : False,
          'lr' : 0.001,
          'momentum': 0.5,
          'optimizer': optim.SGD,
          'activation': F.relu}

  torch.manual_seed(cfg['seed'])
  train_loader, test_loader = get_mnist_loaders(cfg['train_batch_size'], cfg['test_batch_size'])
  model = Net(cfg['activation']).to(device)
  optimizer = cfg['optimizer'](model.parameters(), lr=cfg['lr'])
  for epoch in range(1, cfg['n_epochs'] + 1):
      train(cfg['log_interval'], model, train_loader, optimizer, epoch)
      test_accuracy = test(model, test_loader)

  if cfg['save_model']:
      torch.save(model.state_dict(), "mnist_cnn.pt")

  return test_accuracy

if __name__ == '__main__':
  train_mnist()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302195
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.233102
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.126371
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.926036
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.729658
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.545980
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.221664
Train Epoch: 1 [44800/60000 (75%)]	Loss: 1.034389
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.638690
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.760206

Test set: Average loss: 0.5930, Accuracy: 8636/10000 (86%)



In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 7.9 MB/s 
     |████████████████████████████████| 209 kB 45.6 MB/s 
     |████████████████████████████████| 81 kB 10.0 MB/s 
     |████████████████████████████████| 78 kB 8.1 MB/s 
     |████████████████████████████████| 112 kB 47.0 MB/s 
     |████████████████████████████████| 50 kB 7.8 MB/s 
     |████████████████████████████████| 147 kB 52.1 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=0139060f23cab7175b4a985773e0fcfdbbf8805d7ba7f8c2de4df9e02211ef32
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import optuna
from optuna.trial import TrialState

def objective(trial): # defined the function

    # Generate the model.

    # Generate the optimizers.
    # optimizer_name = trial.suggest_categorical("optimizer", ["SGD","Adam"])
    # scheduler_name=trial.suggest_categorical ('scheduler',['OneCycleLR'])
    # decayrate=trial.suggest_float ('decayrate',0.1,0.95)
    # maxlr=trial.suggest_float ('maxlr',0.5e-1,1e-1)
    lr = trial.suggest_float("lr",1e-5,1e-1) # you choose a lr value automatically
    epoch = trial.suggest_int("epoch",1,3)
    # optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    # scheduler= getattr(optim.lr_scheduler,scheduler_name)(optimizer,maxlr,epochs=30,steps_per_epoch=100)




    cfg = { 'device' : "cuda" if torch.cuda.is_available() else "cpu",
        'train_batch_size' : 64,
        'test_batch_size' : 1000,
        'n_epochs' : epoch,
        'seed' : 0,
        'log_interval' : 100,
        'save_model' : False,
        # 'lr' : 0.001,
        'momentum': 0.5,
        'optimizer': optim.SGD,
        'activation': F.relu}

    torch.manual_seed(cfg['seed'])
    train_loader, test_loader = get_mnist_loaders(cfg['train_batch_size'], cfg['test_batch_size'])
    model = Net(cfg['activation']).to(device)
    optimizer = cfg['optimizer'](model.parameters(), lr=lr) #here u train the n/w with chosen lr value
    for epoch in range(1, cfg['n_epochs'] + 1):
        train(cfg['log_interval'], model, train_loader, optimizer, epoch)
        test_accuracy = test(model, test_loader)
        trial.report(test_accuracy, epoch) #here you report the perf. of the n/e on chosen lr value.

          # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return test_accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=1,n_jobs=1)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2022-11-10 11:19:07,343] A new study created in memory with name: no-name-841b6430-105a-440b-a131-f3f90d4a909d
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1331: UserWarning:

dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).



Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302195
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.752366
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.236180
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.434116
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.257617
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.245193
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.139329
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.110141
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.132032
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.070115

Test set: Average loss: 0.0532, Accuracy: 9831/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.038956
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.032377
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.072943
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.040281
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.084728
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.068643
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.098661
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.098750
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.115483
T

[I 2022-11-10 11:23:53,202] Trial 0 finished with value: 98.58 and parameters: {'lr': 0.08061531158193755, 'epoch': 2}. Best is trial 0 with value: 98.58.



Test set: Average loss: 0.0406, Accuracy: 9858/10000 (99%)

Study statistics: 
  Number of finished trials:  1
  Number of pruned trials:  0
  Number of complete trials:  1
Best trial:
  Value:  98.58
  Params: 
    lr: 0.08061531158193755
    epoch: 2


In [ ]:
from optuna.visualization import plot_optimization_history
plot_optimization_history(study)

In [ ]:
from optuna.visualization import plot_intermediate_values

plot_intermediate_values(study)